In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# 1. 이미지 경로 지정 및 로드
image_folder_path = '/home/elicer/FIshing_vessel-1/imgdata'  # 실제 경로로 변경 필요
image_size = (256, 256)

# 이미지 로드, 크기 조정 및 정규화
image_files = [os.path.join(image_folder_path, file) for file in os.listdir(image_folder_path) if file.lower().endswith(('.jpg', '.png', '.jpeg'))]
images = []
for image_file in image_files:
    img = cv2.imread(image_file)
    if img is not None:
        img = cv2.resize(img, image_size)
        img = img / 255.0  # 정규화
        images.append(img)

# NumPy 배열로 변환
images = np.array(images)

# 2. CSV 파일 읽기 및 타겟 지정
csv_file_path = '/home/elicer/FIshing_vessel-1/preprocessed_whole_dataset.csv'
data = pd.read_csv(csv_file_path)

# 숫자로 된 타겟만 사용
target_columns = ['vessel_type', 'state']  # 숫자 컬럼을 확인하여 변경 필요
targets = data[target_columns].values

# 다중 분류를 위한 원-핫 인코딩
num_classes = [len(np.unique(data[column])) for column in target_columns]
targets = [to_categorical(targets[:, i], num_classes=num_classes[i]) for i in range(len(target_columns))]
targets = np.hstack(targets)

# 3. CNN 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(sum(num_classes), activation='softmax')  # 다중 분류를 위한 출력층
])

# 4. 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 5. 모델 학습
history = model.fit(images, targets, epochs=200, batch_size=32)

# 6. 모델 저장
model.save('fishing_vessel_model.h5')


2024-10-21 06:10:20.093875: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-21 06:10:20.107967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 06:10:20.125487: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 06:10:20.130732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 06:10:20.143670: I tensorflow/core/platform/cpu_feature_guar

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1596
'y' sizes: 1581
